In [34]:
pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network
from dotenv import load_dotenv

In [36]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [37]:
import nest_asyncio

nest_asyncio.apply()

In [38]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set LLM (OpenAI)

In [39]:
load_dotenv()  # Charge les variables depuis le fichier .env
api_key = os.getenv("OPENAI_API_KEY_UP")
# Modifier ou ajouter une variable d'environnement
os.environ["OPENAI_API_KEY"] = api_key

In [40]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings

In [41]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [42]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [43]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
assistant: As a data governance consultant, I don't have personal preferences or feelings. However, I can tell you about some popular data tools that are widely used in the industry for various purposes:

1. **Collibra**: Known for its data governance and cataloging capabilities, Collibra helps organizations manage their data assets and ensure compliance with data policies.

2. **Informatica**: Offers a suite of data management tools, including data integration, data quality, and master data management, which are essential for maintaining data governance.

3. **Alation**: A data catalog tool that helps organizations discover, understand, and manage their data a

# Load storage contexts

## Load vector storage context

In [44]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

## Load knowledge graph storage context

In [45]:
graph_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="knowledge_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="knowledge_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="knowledge_graph"),
)

## Load onto graph storage context


In [46]:
onto_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="onto_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="onto_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="onto_graph"),
)

In [47]:
print(onto_storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x289fcfc70>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x289ffbeb0>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={})), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x289fce6e0>, property_graph_store=None)


# Load index

## Load vector index

In [48]:
simple_index = load_index_from_storage(vector_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


### Set retriever

In [49]:
simple_query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [ ]:
simple_rag_retriever = simple_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = simple_query_engine.query(
    "What is open model ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [51]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, voici un plan structuré :

1. **Compréhension des Données**
   - Identifier les sources de données disponibles (historique des prêts, informations démographiques, etc.).
   - Définir l'objectif de la prédiction : comprendre les facteurs qui influencent le défaut de paiement.

2. **Nettoyage des Données**
   - Supprimer ou corriger les données inexactes ou les doublons.
   - Remplir les données manquantes et uniformiser les formats (par exemple, dates).

3. **Exploration des Données**
   - Analyser les tendances et les relations entre les variables.
   - Utiliser des visualisations pour identifier des modèles potentiels.

4. **Sélection des Caractéristiques**
   - Choisir les variables pertinentes qui pourraient influencer le défaut de paiement.
   - Évaluer l'importance de chaque caractéristique à l'aide de techniques statistiques.

5. **Modélisation Prédictive**
   - Choisir un modèle de machine learning approprié (régression logistique, arbres de décision, etc.).
   - Diviser les données en ensembles d'entraînement et de test.

6. **Évaluation du Modèle**
   - Tester le modèle sur l'ensemble de test pour évaluer sa précision.
   - Utiliser des métriques comme l'accuracy, le rappel, et la précision pour mesurer la performance.

7. **Interprétation et Communication**
   - Interpréter les résultats pour comprendre les facteurs clés du défaut de paiement.
   - Créer des visualisations claires pour communiquer les résultats aux parties prenantes.

8. **Amélioration et Itération**
   - Recueillir des retours des parties prenantes et ajuster le modèle si nécessaire.
   - Répéter le processus pour améliorer la précision et la fiabilité des prédictions.

9. **Mise en Œuvre**
   - Intégrer le modèle dans le système de décision de l'entreprise.
   - Assurer une surveillance continue pour maintenir la performance du modèle. 

Ce plan permet de structurer le processus de prédiction de défaut de paiement en utilisant une approche méthodique et axée sur les données.</b>

## Load graph index

In [52]:
graph_index = load_index_from_storage(graph_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [53]:
nx_graph = graph_index.get_networkx_graph()

In [54]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 61


In [55]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 93 nodes and 61 edges
Nodes: ['Exactitude', 'Justes et cohérentes', 'Complétude', 'Complet', 'Nettoyage', 'Erreurs', 'Normalisation', 'Formats', 'Florence nightingale', 'Infirmière britannique', "Présentation visuelle de l'information", 'Dataviz project', 'Relation', 'Bibliothèque', 'Description', 'Inputs', 'Visualisation', 'Objectif', 'Message', 'Volume', 'Quantité massive de données', 'Vélocité', 'Vitesse de génération de données', 'Base de données relationnelles', 'Données structurées', 'Outils comme sql', 'Base de données non relationnelles', 'Données non structurées', 'Tirer', 'Données', 'Format prédéfini', 'Texte libre', 'Claire et standardisée', 'Passer', 'Interprétation', 'Data visualisation', 'Données brutes en graphiques', 'Information claire et compréhensible', 'Processus', 'Exploitables', 'Dataviz', 'Data visualization', 'Interactive data visualization', 'Edward rolf tufte', 'Professor at yale university', 'Léonard de vinci des données" by new york times', 'Théop

### Set retriever

In [56]:
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [ ]:
graph_rag_retriever = graph_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = graph_query_engine.query(
    "What is open model ?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 6d25ed5c-5caa-4dd3-ae7d-04c06cda84f3: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 6d25ed5c-5caa-4dd3-ae7d-04c06cda84f3: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 6d25ed5c-5caa-4dd3-ae7d-04c06cda84f3: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 6d25ed5c-5caa-4dd3-ae7d-04c06cda84f3: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f92fdb9222e4: Ajout d'éléments dynamiques pour permettre aux [[utilisateur final]]s d'explo...
> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f92fdb9222e4: Ajout d'éléments dynamiques pour permettre aux [[utilisateur final]]s d'explo...
> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f

In [58]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, il est essentiel de suivre un plan structuré qui inclut la collecte, le traitement et l'analyse des données. Voici un plan détaillé :

1. **Collecte des données :**
   - Rassembler des données historiques sur les clients, y compris les informations démographiques, les antécédents de crédit, les revenus, les dettes existantes, etc.
   - Inclure des métadonnées pertinentes pour chaque client, comme la date de la demande de prêt et le type de prêt.

2. **Préparation des données :**
   - Organiser les données de manière claire et standardisée, idéalement dans un tableau avec des lignes et des colonnes.
   - Nettoyer les données pour éliminer les valeurs manquantes ou aberrantes.
   - Transformer les données non structurées, comme les notes de service ou les commentaires des clients, en données utilisables à l'aide de techniques de traitement du langage naturel.

3. **Exploration des données :**
   - Utiliser des visualisations de données pour comprendre les tendances et les distributions. Par exemple, des histogrammes pour la répartition des scores de crédit ou des diagrammes circulaires pour les proportions de défauts par catégorie de revenu.
   - Identifier les biais de perception potentiels dans les données.

4. **Modélisation :**
   - Sélectionner un modèle de machine learning approprié pour la prédiction, comme la régression logistique, les arbres de décision ou les réseaux de neurones.
   - Diviser les données en ensembles d'entraînement et de test pour évaluer la performance du modèle.

5. **Évaluation du modèle :**
   - Utiliser des métriques d'évaluation comme l'accuracy, le rappel, et la précision pour mesurer la performance du modèle.
   - Ajuster les hyperparamètres du modèle pour améliorer les résultats.

6. **Mise en œuvre :**
   - Déployer le modèle dans un environnement de production où il peut être utilisé pour prédire les défauts de paiement en temps réel.
   - Intégrer des éléments dynamiques pour permettre aux utilisateurs finaux d'explorer les prédictions et d'accéder à différents niveaux d'informations.

7. **Surveillance et amélioration continue :**
   - Surveiller la performance du modèle en continu et le mettre à jour avec de nouvelles données pour maintenir sa précision.
   - Recueillir des retours d'expérience des utilisateurs finaux pour améliorer l'interface et l'accessibilité des résultats.

Ce plan permet de structurer le processus de prédiction de défauts de paiement de manière efficace et compréhensible pour tous les utilisateurs, y compris ceux avec des handicaps.</b>

## Load onto index

In [59]:
onto_storage_context = StorageContext.from_defaults(persist_dir="onto_graph")
# Load the PropertyGraphIndex from the storage context

onto_index = load_index_from_storage(onto_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [60]:
onto_index.property_graph_store.save_networkx_graph(
    name="OntoGraph.html"
)

### Set retriever

In [61]:
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=8,
)

### Test retriever

In [62]:
onto_rag_retriever = onto_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = onto_query_engine.query(
    "What is open model ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [63]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, voici un plan structuré :

1. **Collecte et Préparation des Données :**
   - Rassembler les données historiques des clients, y compris les informations démographiques, les antécédents de crédit, les revenus, etc.
   - Nettoyer les données pour éliminer les valeurs manquantes ou aberrantes.
   - Transformer les données en un format approprié pour l'analyse, par exemple en normalisant les variables numériques.

2. **Exploration des Données :**
   - Utiliser des techniques de visualisation de données pour identifier les tendances et les relations entre les variables.
   - Analyser les corrélations entre les caractéristiques des clients et les défauts de paiement.

3. **Sélection des Caractéristiques :**
   - Identifier les variables les plus pertinentes qui influencent le risque de défaut.
   - Utiliser des techniques de réduction de dimensionnalité si nécessaire, comme l'analyse en composantes principales (PCA).

4. **Choix du Modèle :**
   - Sélectionner un modèle de machine learning approprié, tel que la régression logistique, les arbres de décision, ou les forêts aléatoires.
   - Considérer l'utilisation de modèles plus avancés comme les réseaux de neurones ou les modèles de gradient boosting si les données sont volumineuses et complexes.

5. **Entraînement et Validation du Modèle :**
   - Diviser les données en ensembles d'entraînement et de test.
   - Entraîner le modèle sur l'ensemble d'entraînement.
   - Valider le modèle en utilisant l'ensemble de test pour évaluer sa précision et sa capacité de généralisation.

6. **Évaluation du Modèle :**
   - Utiliser des métriques d'évaluation telles que l'accuracy, le F1-score, la précision, et le rappel pour mesurer la performance du modèle.
   - Effectuer une analyse des erreurs pour comprendre les cas de défauts mal prédits.

7. **Optimisation et Ajustement :**
   - Ajuster les hyperparamètres du modèle pour améliorer sa performance.
   - Envisager l'utilisation de techniques d'ensemble pour combiner plusieurs modèles et améliorer la robustesse des prédictions.

8. **Déploiement et Surveillance :**
   - Déployer le modèle dans un environnement de production pour prédire les défauts en temps réel.
   - Mettre en place un système de surveillance pour suivre la performance du modèle et effectuer des mises à jour si nécessaire.

9. **Interprétation et Communication des Résultats :**
   - Présenter les résultats de manière compréhensible pour les parties prenantes, en utilisant des visualisations claires.
   - Discuter des implications des prédictions pour la gestion des risques de crédit.

Ce plan fournit une approche systématique pour développer un modèle prédictif de défaut de paiement sur un prêt bancaire.</b>

# Visualize knowledge graph

In [64]:
nx_graph = graph_index.get_networkx_graph()

In [65]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 61


In [66]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 93 nodes and 61 edges
Nodes: ['Exactitude', 'Justes et cohérentes', 'Complétude', 'Complet', 'Nettoyage', 'Erreurs', 'Normalisation', 'Formats', 'Florence nightingale', 'Infirmière britannique', "Présentation visuelle de l'information", 'Dataviz project', 'Relation', 'Bibliothèque', 'Description', 'Inputs', 'Visualisation', 'Objectif', 'Message', 'Volume', 'Quantité massive de données', 'Vélocité', 'Vitesse de génération de données', 'Base de données relationnelles', 'Données structurées', 'Outils comme sql', 'Base de données non relationnelles', 'Données non structurées', 'Tirer', 'Données', 'Format prédéfini', 'Texte libre', 'Claire et standardisée', 'Passer', 'Interprétation', 'Data visualisation', 'Données brutes en graphiques', 'Information claire et compréhensible', 'Processus', 'Exploitables', 'Dataviz', 'Data visualization', 'Interactive data visualization', 'Edward rolf tufte', 'Professor at yale university', 'Léonard de vinci des données" by new york times', 'Théop

# (Simple) Query the vectors

In [67]:
query = "What is open model ?"
query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [68]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, voici un plan structuré :

1. **Compréhension des Données**
   - Identifier les sources de données disponibles (historique des prêts, informations démographiques, etc.).
   - Définir l'objectif de la prédiction : identifier les clients à risque de défaut.

2. **Nettoyage des Données**
   - Supprimer ou corriger les données inexactes ou les doublons.
   - Remplir les données manquantes et uniformiser les formats (par exemple, dates).

3. **Exploration des Données**
   - Analyser les tendances et les relations entre les variables.
   - Utiliser des visualisations pour comprendre les distributions et les corrélations.

4. **Sélection des Caractéristiques**
   - Identifier les variables pertinentes qui influencent le risque de défaut (revenu, historique de crédit, etc.).
   - Évaluer l'importance des caractéristiques à l'aide de techniques statistiques.

5. **Modélisation Prédictive**
   - Choisir un modèle de machine learning approprié (régression logistique, arbres de décision, etc.).
   - Entraîner le modèle sur un ensemble de données d'entraînement.

6. **Évaluation du Modèle**
   - Tester le modèle sur un ensemble de données de test pour évaluer sa précision.
   - Utiliser des métriques telles que l'accuracy, le rappel, et la précision.

7. **Interprétation et Visualisation**
   - Créer des visualisations pour expliquer les prédictions du modèle.
   - Assurer que les visualisations sont compréhensibles pour les parties prenantes.

8. **Mise en Production**
   - Intégrer le modèle dans le système de décision de l'entreprise.
   - Mettre en place un processus de surveillance pour évaluer la performance continue du modèle.

9. **Retour d'Expérience et Amélioration**
   - Recueillir des retours des utilisateurs finaux et des parties prenantes.
   - Apporter des modifications et des améliorations au modèle si nécessaire. 

Ce plan permet de structurer le processus de prédiction du défaut de paiement en utilisant une approche méthodique et centrée sur les données.</b>

# (Simple) Query the knowledge graph 

In [69]:
query = "What is open model ?"
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 6d25ed5c-5caa-4dd3-ae7d-04c06cda84f3: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 6d25ed5c-5caa-4dd3-ae7d-04c06cda84f3: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 6d25ed5c-5caa-4dd3-ae7d-04c06cda84f3: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 6d25ed5c-5caa-4dd3-ae7d-04c06cda84f3: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f92fdb9222e4: Ajout d'éléments dynamiques pour permettre aux [[utilisateur final]]s d'explo...
> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f92fdb9222e4: Ajout d'éléments dynamiques pour permettre aux [[utilisateur final]]s d'explo...
> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f

In [70]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, il est essentiel de suivre un plan structuré qui inclut la préparation des données, la sélection d'un modèle de prédiction, et l'évaluation des résultats. Voici un plan détaillé :

1. **Compréhension du problème et des données :**
   - Définir clairement l'objectif de la prédiction : identifier les clients susceptibles de faire défaut.
   - Identifier les caractéristiques des données disponibles, y compris les métadonnées pertinentes (par exemple, historique de crédit, revenu, etc.).

2. **Préparation des données :**
   - Nettoyer les données en supprimant ou corrigeant les inexactitudes et les doublons.
   - Remplir les données manquantes et uniformiser les formats (par exemple, dates).
   - Organiser les données de manière claire et standardisée, souvent dans un tableau avec des lignes et des colonnes.

3. **Exploration et visualisation des données :**
   - Utiliser des visualisations pour comprendre les tendances et les relations dans les données (par exemple, histogrammes pour la répartition des scores de crédit).
   - Identifier les variables les plus pertinentes pour la prédiction.

4. **Sélection et entraînement du modèle :**
   - Choisir un modèle de machine learning adapté (par exemple, régression logistique, arbres de décision).
   - Diviser les données en ensembles d'entraînement et de test.
   - Entraîner le modèle sur l'ensemble d'entraînement.

5. **Évaluation du modèle :**
   - Tester le modèle sur l'ensemble de test pour évaluer sa précision et sa capacité à prédire les défauts.
   - Utiliser des métriques d'évaluation comme l'accuracy, le rappel, et la précision.

6. **Interprétation et communication des résultats :**
   - Interpréter les résultats pour comprendre les facteurs qui influencent le défaut de paiement.
   - Créer des visualisations compréhensibles pour communiquer les résultats aux parties prenantes, y compris les utilisateurs finaux.

7. **Mise en œuvre et suivi :**
   - Intégrer le modèle dans le système de décision de l'entreprise.
   - Mettre en place un suivi pour évaluer la performance du modèle dans le temps et ajuster si nécessaire.

Ce plan permet de structurer le processus de prédiction de manière efficace et de s'assurer que les résultats sont fiables et utiles pour la prise de décision.</b>

# (Simple) Query the onto graph 

In [71]:
query = "What is open model ?"
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=2,
)

response = onto_query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [72]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, voici un plan structuré :

1. **Collecte et Préparation des Données :**
   - Rassembler les données historiques des clients, y compris les informations démographiques, les antécédents de crédit, les revenus, les dettes existantes, etc.
   - Nettoyer les données pour éliminer les valeurs manquantes ou aberrantes.
   - Créer des variables dérivées si nécessaire, comme le ratio dette/revenu.

2. **Exploration des Données :**
   - Utiliser des outils de visualisation de données pour explorer les relations entre les différentes variables et le défaut de paiement.
   - Identifier les tendances et les modèles potentiels dans les données.

3. **Sélection des Caractéristiques :**
   - Choisir les caractéristiques les plus pertinentes qui influencent le défaut de paiement.
   - Utiliser des techniques comme l'analyse de corrélation ou la sélection de caractéristiques basée sur l'importance.

4. **Choix du Modèle :**
   - Sélectionner un modèle de machine learning approprié, tel que la régression logistique, les arbres de décision, ou les forêts aléatoires.
   - Justifier le choix du modèle en fonction de la nature des données et des objectifs de prédiction.

5. **Entraînement et Validation du Modèle :**
   - Diviser les données en ensembles d'entraînement et de test.
   - Entraîner le modèle sur l'ensemble d'entraînement.
   - Valider le modèle en utilisant l'ensemble de test pour évaluer sa précision et sa capacité de généralisation.

6. **Évaluation du Modèle :**
   - Utiliser des métriques d'évaluation telles que l'accuracy, le F1-score, la précision, et le rappel pour mesurer la performance du modèle.
   - Effectuer une analyse des erreurs pour comprendre les cas où le modèle se trompe.

7. **Optimisation et Ajustement :**
   - Ajuster les hyperparamètres du modèle pour améliorer sa performance.
   - Envisager l'utilisation de techniques d'ensemble pour combiner plusieurs modèles et améliorer la précision.

8. **Déploiement et Surveillance :**
   - Déployer le modèle dans un environnement de production.
   - Mettre en place un système de surveillance pour suivre la performance du modèle au fil du temps et effectuer des mises à jour si nécessaire.

9. **Interprétation et Communication des Résultats :**
   - Interpréter les résultats du modèle pour les parties prenantes.
   - Utiliser des visualisations pour expliquer les prédictions et les facteurs influents.

Ce plan permet de structurer le processus de prédiction du défaut de paiement d'un client de manière méthodique et efficace.</b>

## (Node retriever)

In [73]:
retriever = onto_index.as_retriever(
    include_text=True,  # include source text, default True
)

nodes = retriever.retrieve("What is open model ?")


for node in nodes:
    print(node.text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Here are some facts extracted from the provided text:

Gérer qui peut avoir accès -> EXAMPLE_OF -> Ajout d'éléments dynamiques
Ajout d'éléments dynamiques -> ENABLES -> créer une relation à la donnée

Ajout d'éléments dynamiques pour permettre aux [[utilisateur final]]s d'explorer les [[données]] de différentes manières
Permet de créer une relation à la donnée 

Exemple : 
- Gérer des filtres
- Pouvoir afficher des détails
- Avoir accès à différents niveaux d'informations, ou gérer qui peut avoir accès à quel niveau d'information 

#concept
Here are some facts extracted from the provided text:

théophile -> BELIEVES -> donnée liée à outil informatique

> Perso,

# Have a real chat with your data

## Set up the engines

### Vector engines

In [74]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a specialist about open models and you nurture a knowledge base on the subject"
        "Be precise."
        "."
    ),
    verbose=False,
)

### Graph engines

In [75]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    verbose=False,
)

### Onto engines

In [76]:
#onto_chat_engine = onto_index.query_engine(
#    chat_mode="condense_plus_context",
#    llm=llm
#)

## Generate

In [99]:
graph_chat_engine.reset()

In [100]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a specialist about open models and you nurture a knowledge base on the subject"
        "Be precise on concepts."
        " "
    ),
    verbose=False,
)

In [101]:
response_stream = graph_chat_engine.stream_chat("""Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. """)

INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. 
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. 
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. 
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Access. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. 
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https:

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f92fdb9222e4: Ajout d'éléments dynamiques pour permettre aux [[utilisateur final]]s d'explo...
> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f92fdb9222e4: Ajout d'éléments dynamiques pour permettre aux [[utilisateur final]]s d'explo...
> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f92fdb9222e4: Ajout d'éléments dynamiques pour permettre aux [[utilisateur final]]s d'explo...
> Querying with idx: 87ca8359-7aa1-4fac-9e8a-f92fdb9222e4: Ajout d'éléments dynamiques pour permettre aux [[utilisateur final]]s d'explo...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0

In [102]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
### Definition
Open Access (OA) refers to the practice of providing unrestricted online access to scholarly research outputs, such as journal articles, conference papers, theses, and book chapters. The primary goal of Open Access is to make research freely available to anyone with internet access, thereby promoting the dissemination and exchange of knowledge without financial, legal, or technical barriers.

### Presentation
Open Access is a transformative approach to scholarly publishing that challenges traditional subscription-based models. It is characterized by the removal of paywalls and the provision of free access to research outputs. Open Access can be a

In [103]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

In [104]:
section_modele = """ 


"""

In [105]:
response_stream = graph_chat_engine.stream_chat("""Based on your knowledge and your vision of the world, create a description of Open Business. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. Imite le style d'écriture de la {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Based on your knowledge and your vision of the world, create a description of Open Business. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Imitate the writing style of the Open Access section model.
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Business. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Imitate the writing style of the Open Access section model.
Condensed question: Based on your knowledge and your vision of the world, cre

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: a7f04498-74a7-4a8f-ba33-35f9cec1cfbb: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
> Querying with idx: a7f04498-74a7-4a8f-ba33-35f9cec1cfbb: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
> Querying with idx: a7f04498-74a7-4a8f-ba33-3

In [106]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
### Définition
L'Open Business désigne un modèle d'entreprise qui repose sur la transparence, la collaboration et le partage ouvert d'informations, de ressources et de processus. Ce concept vise à créer de la valeur en impliquant un large éventail de parties prenantes, y compris les clients, les partenaires et même les concurrents, dans le développement et l'amélioration des produits et services.

### Présentation
L'Open Business se distingue par son approche participative et inclusive, qui remet en question les structures hiérarchiques traditionnelles et les pratiques commerciales cloisonnées. Les entreprises adoptant ce modèle favorisent la co-création et l'i

In [107]:
response_stream = graph_chat_engine.stream_chat("""Based on your knowledge and your vision of the world, create a description of Open Banking. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. Imite le style d'écriture de la {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Based on your knowledge and your vision of the world, create a description of Open Banking. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Please imitate the writing style of the previous sections.
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Banking. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Please imitate the writing style of the previous sections.
Condensed question: Based on your knowledge and your vision of the world, create 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: a7f04498-74a7-4a8f-ba33-35f9cec1cfbb: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
> Querying with idx: a7f04498-74a7-4a8f-ba33-35f9cec1cfbb: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
> Querying with idx: a7f04498-74a7-4a8f-ba33-3

In [108]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
### Définition
L'Open Banking désigne un système bancaire qui permet le partage sécurisé des données financières des clients entre différentes institutions financières et prestataires de services tiers, avec le consentement explicite des clients. Ce concept vise à favoriser l'innovation et la concurrence dans le secteur financier en ouvrant l'accès aux données bancaires traditionnellement détenues par les banques.

### Présentation
L'Open Banking se caractérise par l'utilisation d'interfaces de programmation d'applications (API) qui permettent aux banques et aux prestataires tiers d'accéder aux données financières des clients de manière sécurisée et standardisé

In [109]:
response_stream = graph_chat_engine.stream_chat("""Based on your knowledge and your vision of the world, create a description of Open Design. It needs 5 parts : a definition, a presentation, a usage description, a landscape and history. Imite le style d'écriture de la {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Based on your knowledge and your vision of the world, create a description of Open Design. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Please imitate the writing style of the previous sections.
Condensed question: Based on your knowledge and your vision of the world, create a description of Open Design. It needs 5 parts: a definition, a presentation, a usage description, a landscape, and history. Please imitate the writing style of the previous sections.
Condensed question: Based on your knowledge and your vision of the world, create a 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
> Querying with idx: 9d1d3368-aed1-43da-b64d-0bdba0439f8b: Comprendre l'objectif de la visualisation. Avoir une première idée de ce que ...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: a7f04498-74a7-4a8f-ba33-35f9cec1cfbb: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
> Querying with idx: a7f04498-74a7-4a8f-ba33-35f9cec1cfbb: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
> Querying with idx: a7f04498-74a7-4a8f-ba33-3

In [110]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
### Définition
L'Open Design désigne une approche de conception qui encourage le partage libre et collaboratif des plans, des idées et des processus de création. Ce concept permet à des designers, ingénieurs et créateurs du monde entier de contribuer, modifier et améliorer des projets de manière ouverte, souvent en utilisant des licences qui permettent la redistribution et la modification des œuvres.

### Présentation
L'Open Design se caractérise par sa transparence et son inclusivité, permettant à toute personne intéressée de participer au processus de conception. Les projets d'Open Design sont souvent hébergés sur des plateformes en ligne où les fichiers de c

In [111]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

## Sum-up

In [112]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"/Users/arthursarazin/Documents/coreandgraphs/graphandopenmodels/output.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 